# \## NOTE
this file takes as input the file made in data_cleaning, and creates a new file further cleaned
# \###

## Data Exploration

`Author: Andrea Zanon`

In [1]:
# numpy
import numpy as np

# pandas
import pandas as pd

# identify digits from text
from string import digits

# regular expressions
import re

# nltk, useful to clean text (may be needed to donwload files if using for first time)
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
#nltk.download('punkt')
#nltk.download('stopwords')
#nltk.download('wordnet')

# detect language from text
import langid

In [2]:
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')


[nltk_data] Downloading package punkt to /Users/victor/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/victor/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /Users/victor/nltk_data...


True

In [3]:
data = pd.read_csv("/home/azanon/ML_NOAC_NOVA_Extraction.csv")
data.head()

,CaseNumber,Type,Topics__c,TeamName__c,RequesterEmail__c,EmailTemplateSubjectDispute__c,LastEmailCCAddress__c,SuppliedEmail,LastIncomingEmail__c,attributes.type,attributes.url,Contact.attributes.type,Contact.attributes.url,Contact.Email,Contact
0,100124562,RTCU,T039,US-NOR-CMA-BKG-PREFERRED,NaN,NAM4999926A Case #100124562,op@easywayintl.com,NaN,"<div>\n<div>\n<div dir=""ltr"" style=""color: rgb...",Case,/services/data/v42.0/sobjects/Case/5005p00002e...,Contact,/services/data/v42.0/sobjects/Contact/0035p000...,usa.wcportops@cma-cgm.com,NaN
1,100128892,CRFC,T038,US-NOR-CMA-CS-EXP-PREFERRED,NaN,NAM5085287 Case #100128892,NaN,NaN,NaN,Case,/services/data/v42.0/sobjects/Case/5005p00002e...,NaN,NaN,NaN,NaN
2,100123788,BCON,T847,US-NOR-CMA-CS-EXP-PREFERRED,NaN,NAM4993560 Case #100123788,NaN,NaN,NaN,Case,/services/data/v42.0/sobjects/Case/5005p00002e...,Contact,/services/data/v42.0/sobjects/Contact/0030N000...,ds@srinc.us.com,NaN
3,100080371,CRFC,T036,US-BNA-CMA-CS-IMP-DIVERSIONS,NaN,MOGELIJKE COD'S F119086 // RTM Case #100080371,janneke.van.leeuwen@fclmarine.nl; executivedes...,NaN,"<div class=""WordSection1"">\n<p class=""MsoNorma...",Case,/services/data/v42.0/sobjects/Case/5005p00002e...,Contact,/services/data/v42.0/sobjects/Contact/0030N000...,janneke.van.leeuwen@fclmarine.nl,NaN
4,100095395,BREQ,T035,US-NOR-CMA-BKG-PREFERRED,tina@generallogistics.net,AMEND NAM5088900 [ ref:_00D Case #100095395,bookings@generallogistics.net,tina@generallogistics.net,"<div class=""WordSection1"">\n<p class=""MsoNorma...",Case,/services/data/v42.0/sobjects/Case/5005p00002e...,Contact,/services/data/v42.0/sobjects/Contact/0030N000...,tina@generallogistics.net,NaN


In [5]:
# Rename LastIncomingEmail__c in LastIncomingEmail
data = data.rename(columns={"LastIncomingEmail__c": "LastIncomingEmail"})

In [3]:
# work on copy on data
df = data.copy()

In [7]:
df.isnull().sum()

CaseNumber                      0
Type                            0
Topics                       1078
TeamName                        0
RequesterEmail               5076
EmailObject                  1417
LastEmailCCAddress          90924
AttributesURL                   0
ContactAttributesURL        24360
ContactEmail                  344
LastIncomingEmailContent       17
LastEmailCCAddressCount         0
dtype: int64

In [8]:
# From LastEmailCCAddress get whether cma-cgm is cc'd or not
df['CMA_in_cc'] = df['LastEmailCCAddress'].str.find('cma-cgm') > -1

In [9]:
# Remove all numbers from emails, they are not relevant
df["LastIncomingEmailContent"] = df["LastIncomingEmailContent"].str.translate(str.maketrans('', '', digits))

In [10]:
# re.findall("\n(.*)\n", s)
# this returns the email sentence by sentence

In [11]:
df.head()

,CaseNumber,Type,Topics,TeamName,RequesterEmail,EmailObject,LastEmailCCAddress,AttributesURL,ContactAttributesURL,ContactEmail,LastIncomingEmailContent,LastEmailCCAddressCount,CMA_in_cc
0,100124562,0,T039,US-NOR-CMA-BKG-PREFERRED,NaN,NAM4999926A,op@easywayintl.com,5005p00002evRjHAAU,0035p00003aZAdZAAW,usa.wcportops@cma-cgm.com,\n\n\nThank you Jessica! Await your further on...,1,False
1,100080371,0,T036,US-BNA-CMA-CS-IMP-DIVERSIONS,NaN,MOGELIJKE COD'S F119086 // RTM,janneke.van.leeuwen@fclmarine.nl; executivedes...,5005p00002evWsuAAE,0030N00003N771RQAR,janneke.van.leeuwen@fclmarine.nl,"\nHi Nese,\n \nInmiddels de aangepaste bl’s al...",5,True
2,100095395,0,T035,US-NOR-CMA-BKG-PREFERRED,tina@generallogistics.net,AMEND NAM5088900 [ ref:_00D,bookings@generallogistics.net,5005p00002evX4SAAU,0030N00002x1JqPQAU,tina@generallogistics.net,"\nHello,\nPlease SPLIT subject line booking to...",1,False
3,100097054,0,T847,US-NOR-CMA-BKG-SPECIALTY,bookings@delongcompany.com,FW: CMA CGM - Cut-off date cha,usa.cmaexportactive@usa.cma-cgm.com,5005p00002evXIsAAM,0030N00002tXvvIQAS,bookings@delongcompany.com,"CMA CGM - Cut-off date change at - NEW YORK, N...",1,True
4,100097055,0,T847,US-NOR-CMA-BKG-SPECIALTY,bookings@delongcompany.com,FW: CMA CGM - Cut-off date cha,NaN,5005p00002evXItAAM,0030N00002tXvvIQAS,bookings@delongcompany.com,"CMA CGM - Cut-off date change at - NEW YORK, N...",0,False


In [12]:
# drop rows that have NaN in LastIncomingEmailContent
df = df.drop(df[df["LastIncomingEmailContent"].isnull()].index)

# reindex, otherwise some indexes are missing
df = df.reset_index()

In [13]:
# new feature: how many emails are exchanged in each conversation, counting Sent:
df["CountMailsInConversation"] = [len(re.findall("Sent: .* :To:", str(df["LastIncomingEmailContent"][i]))) for i in range(len(df["LastIncomingEmailContent"]))]

In [ ]:
# extract just text of the first email. I try doing that by cutting off when there is a salutation
# eliminate characters non needed and stopwords

# define lemmatizer
lemmatizer = WordNetLemmatizer()

only_text = []
for i in range(len(df["LastIncomingEmailContent"])):
    
    # get text before one of the salutations
    text = re.split('Best|Regards|Rgds|Warmest|Warmly|Looking forward|Be well|Yours Truly|Sincerely|Booking ref| \
                      From: .* <.*>' , df.loc[i, "LastIncomingEmailContent"], flags=re.IGNORECASE)[0]
    
    # remove if language is not english (replace by NaN and then remove NaN)
    if langid.classify(text) != 'en':
        only_text.append("")
        continue  # go to next iteration
    
    # remove these because we don't need them
    for ch in ['\n', '\xa0']:
        if ch in text:
            text = text.replace(ch, '')
    
    # remove stopwords
    text_tokens = word_tokenize(text)
    text = ' '.join([word for word in text_tokens if not word in stopwords.words()])
    
    # keep everything lowercase
    text = text.lower()
    
    # word lemmatization
    lemma_words = [lemmatizer.lemmatize(o) for o in text.split()]
    text = " ".join(lemma_words)
                 
    only_text.append(text)

In [ ]:
# append to new dataframe column
df['only_text'] = only_text

# remove nan elements
# drop rows that have NaN in only_text
df = df.drop(df[df["only_text"].isnull()].index)

# reindex, otherwise some indexes are missing
df = df.reset_index()

In [ ]:
# save file
df.to_csv('/home/azanon/ML_NOAC_NOVA_Extraction_Cleaned_New.csv', index=False)